In [4]:

from radiant_mlhub import Collection, Dataset

bbox = [28.529402, -1.753992, 30.328979, 1.359627]

re_dir = "./data/radiantEarth"

dataset = Dataset.fetch('ref_landcovernet_af_v1')
print(dataset.collections)
print(dataset.collections.source_imagery)
print(dataset.collections.labels)

collection_filter = dict(
    ref_landcovernet_af_v1_source_sentinel_2=['B02', 'B03', 'B04', 'B08'],
    ref_landcovernet_af_v1_labels=['labels', 'documentation']
)

# collection_filter still downloads catalogs for other collections - dumb
# dataset.download(collection_filter=collection_filter, output_dir="./data/radiantEarth/")

[<Collection id=ref_landcovernet_af_v1_source_sentinel_2>, <Collection id=ref_landcovernet_af_v1_source_sentinel_1>, <Collection id=ref_landcovernet_af_v1_source_landsat_8>, <Collection id=ref_landcovernet_af_v1_labels>]
[<Collection id=ref_landcovernet_af_v1_source_sentinel_2>, <Collection id=ref_landcovernet_af_v1_source_sentinel_1>, <Collection id=ref_landcovernet_af_v1_source_landsat_8>]
[<Collection id=ref_landcovernet_af_v1_labels>]


In [ ]:
from radiant_mlhub import Collection
import tarfile


re_dir = "./data/radiantEarth"

labels_collection = Collection.fetch('ref_landcovernet_af_v1_labels')
sentinel_collection = Collection.fetch('ref_landcovernet_af_v1_source_sentinel_2')

labels_archive = labels_collection.download(f'{re_dir}/labels')
sentinel_archive = sentinel_collection.download(f'{re_dir}/images')


  0%|          | 0/87989.0 [00:00<?, ?M/s]

In [ ]:
labels_tar = tarfile.open(labels_archive, "r:gz")
labels_tar.extractall()
labels_tar.close()

In [ ]:
sentinel_tar = tarfile.open(sentinel_archive, "r:gz")
sentinel_tar.extractall()
sentinel_tar.close()

In [ ]:
# would be cool to build a cloud cover model!
# https://mlhub.earth/data/ref_cloud_cover_detection_challenge_v1
# https://drivendata.co/blog/cloud-cover-benchmark/

# or https://www.azavea.com/blog/2021/08/02/the-azavea-cloud-dataset/